In [1]:
import mlbeats
import pandas as pd
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
import biosppy

In [2]:
filenumber=[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 200, 201, 202, 203, 205, 207, 208, 209, 210, 212, 213, 214, 215, 217, 219, 220, 221, 222, 223, 228, 230, 231, 232, 233, 234]

In [3]:
heart_beat=pd.DataFrame()

In [5]:
for j in filenumber:
    test_file = pd.read_csv("mitbih_database/"+str(j)+".csv")
    ecg_signal = test_file.iloc[:, 1].to_numpy()
    data = []
    with open("mitbih_database/"+str(j)+"annotations.txt", 'r') as data_file:
        for line in data_file:
            item=line.split()
            if len(item)==7:
                item[2]=item[-1]
                item=item[0:7]
                data.append(item[1:3])
            else:
                data.append(item[1:3])
    annotations=pd.DataFrame(data[2:], columns=["Sample#", "Result"])
    rpeak=[]
    for i in range(len(annotations['Sample#'])):
        x = int(annotations['Sample#'][i])
        rpeak.append(x)
    rpeak = np.array(rpeak)
    ecg_signal = nk.ecg.ecg_clean(ecg_signal, sampling_rate=360, method='neurokit')
    epochs, sudo_rpeak=biosppy.signals.ecg.extract_heartbeats(signal=ecg_signal, rpeaks=rpeak, sampling_rate=360, before=0.4, after=0.4)
    
    
    
    
    labels = []
    for k in sudo_rpeak:
        row = annotations[annotations["Sample#"] == str(k)]
        labels.append(row.iloc[:, 1].to_numpy()[0])
    S = {"A", "a", "J", "S"}
    N = {"N", "L", "R", "e", "j"}
    V = {"V", "E"}
    F = {"F"}
    Q = {"/", "f", "Q"}
    for m in range(len(labels)):
        if labels[m] in N:
            labels[m] = "N"
        elif labels[m] in S:
            labels[m] = "S"
        elif labels[m] in V:
            labels[m] = "V"
        elif labels[m] in F:
            labels[m] = "F"
        elif labels[m] in Q:
            labels[m] = "Q"
        else:
            pass

        
      
        
        
    drop_num=[] 
    for n in range(288):
        if n%3 != 0:
            drop_num.append(n)
            
            
        
    epochs = pd.DataFrame(epochs)
    epochs["Result"] = labels
    epochs = epochs.drop(drop_num, axis=1)
    heart_beat = pd.concat([heart_beat, epochs], axis=0)
    print(j)

100
101
102
103
104
105
106
107
108
109
111
112
113
114
115
116
117
118
119
121
122
123
124
200
201
202
203
205
207
208
209
210
212
213
214
215
217
219
220
221
222
223
228
230
231
232
233
234


In [ ]:
print(heart_beat)

In [ ]:
heart_beat.to_csv("beat_extraction.csv")